In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

conn = sqlite3.connect('poker_updated.db')
c = conn.cursor()

In [2]:
total_games = pd.read_sql_query('SELECT player_ID, COUNT(*) AS total_games FROM Hands \
    GROUP BY player_ID;', conn)
preflop_folds = pd.read_sql_query('''SELECT player_ID, COUNT(*) AS num_preflop_folds FROM Actions 
    WHERE round_k = 0 AND action_id = 'f'
    GROUP BY player_ID;''', conn)
preflop_raises = pd.read_sql_query('''SELECT player_ID, COUNT(*) AS num_preflop_raises FROM Actions 
    WHERE round_k = 0 AND action_id = 'r'
    GROUP BY player_ID;''', conn)
all_ins = pd.read_sql_query('''SELECT player_ID, COUNT(*) AS num_all_ins FROM Actions 
    WHERE action_id = 'A'
    GROUP BY player_ID;''', conn)
showdowns = pd.read_sql_query('''SELECT player_ID, COUNT(*) AS num_showdowns FROM Actions \
    WHERE round_k = 3 AND action_id <> 'f'\
    GROUP BY player_ID \
    ORDER BY player_ID;''', conn)
net_gains = pd.read_sql_query('SELECT player_ID, SUM(net_gain) AS total_net_gain FROM Hands \
GROUP BY player_ID;', conn)

#create summany dataframe containing all data
summary = pd.merge(total_games, preflop_folds, on='player_ID')
summary = pd.merge(summary, preflop_raises, on='player_ID')
summary = pd.merge(summary, all_ins, on='player_ID')
summary = pd.merge(summary, showdowns, on='player_ID')

#normalize by number of games each player plays
summary['preflop_fold_rate'] = summary['num_preflop_folds']/summary['total_games']
summary['preflop_raise_rate'] = summary['num_preflop_raises']/summary['total_games']
summary['all_in_rate'] = summary['num_all_ins']/summary['total_games']
summary['showdown_rate'] = summary['num_showdowns']/summary['total_games']

summary = pd.merge(summary, net_gains, on='player_ID')
summary['avg_net_gain'] = summary['total_net_gain']/summary['total_games']
print(summary.head(5))

          player_ID  total_games  num_preflop_folds  num_preflop_raises  \
0     !MushroomMan!           54                 43                   5   
1        0nlyDaNuts          260                199                  30   
2  1000_na_nl25_aga          305                221                  62   
3          10071995           81                 64                  10   
4         100stacks           50                 42                   4   

   num_all_ins  num_showdowns  preflop_fold_rate  preflop_raise_rate  \
0            1              1           0.796296            0.092593   
1            2             10           0.765385            0.115385   
2            1             15           0.724590            0.203279   
3            1              7           0.790123            0.123457   
4            1              3           0.840000            0.080000   

   all_in_rate  showdown_rate  total_net_gain  avg_net_gain  
0     0.018519       0.018519           45.24      0.8

In [135]:
summary = summary[summary['total_net_gain'] > -500]
summary = summary[summary['total_games'] > 100]
print("num_players: ", summary.shape[0])

num_players:  342


In [136]:
X = summary[['preflop_fold_rate', 'preflop_raise_rate', 'all_in_rate', 'showdown_rate']].values
y = summary['avg_net_gain'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [137]:
regressor = LinearRegression() 
regressor.fit(X_train, y_train)
print(regressor.coef_)

[ 0.71972684 -0.57265951  0.15676051  2.55353653]


In [138]:
y_pred = regressor.predict(X_test)
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

In [139]:
print('MSE:', metrics.mean_squared_error(y_test, y_pred))

MSE: 1.1276745040031513


In [140]:
X = summary[['preflop_fold_rate', 'preflop_raise_rate', 'all_in_rate', 'showdown_rate']].values
y = summary['avg_net_gain'].values > 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LogisticRegression() 
regressor.fit(X_train, y_train)
print(regressor.coef_)

y_pred = regressor.predict(X_test)
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(y_pred)

[[ 0.05944242  0.19165255 -0.14009029  0.00297818]]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False]


/Users/liambai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
